In [2]:
#Smart Jukebox
!pip install fuzzywuzzy
import pandas as pd
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import ipywidgets as widgets
import sys

df = pd.read_csv("Spotify_Youtube.csv")

# Normalize the loudness column into [0,1] range
df['Loudness_normalized'] = (df['Loudness'] - df['Loudness'].min())/ (df['Loudness'].max()-df['Loudness'].min())

def get_user_input(prompt):
    user_input = input(f"{prompt}")
    return user_input

#Start
print("Welcome to Smart Jukebox! Answer each question and we'll create a personalized playlist for you.")
create_only_artist_playlist = get_user_input("Would you like to create a playlist that only includes your favorite artist? (yes/no)")

if create_only_artist_playlist.lower() == "yes":
    fav_artist = get_user_input("Who is your favorite artist?")
    num_songs_input = int(get_user_input("How many songs do you want in your playlist?"))
    num_songs = min(num_songs_input, len(df))
    # Fuzzywuzzy allows alternate spellings in the artist name, here the user needs only 75% of artist name
    best_match = process.extractOne(fav_artist, df['Artist'].unique(), scorer=fuzz.partial_ratio)
    if best_match[1] >= 75:
        artist_songs = df[df["Artist"].str.lower() == best_match[0].lower()][["Artist", "Url_youtube","Track"]]
    else:
        print(f"No songs found for {fav_artist}. Please try again.")
        sys.exit()
    playlist = artist_songs.sample(n=num_songs, replace=True)
    if len(playlist) < num_songs:
        print(f"Not enough unique songs found for {fav_artist}. Your playlist only contains {len(playlist)} unique songs.")
    else:
        print("\nYour playlist:")
        for i, song in playlist.iterrows():
            print(f"{song['Track']} by {song['Artist']} {song['Url_youtube']}")
else:
    print("")
    print("Description of each parameter:")
    print("- Danceability: A value of 0.0 is least danceable and 1.0 is most danceable.")
    print("- Energy: Death metal has high energy, while a Bach prelude scores low on the scale.")
    print("- Loudness: the overall loudness of a track in decibels (dB). ")
    print("- Speechiness: The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.")
    print("- Acousticness: 1.0 represents high confidence the track is acoustic.")
    print("- Instrumentalness: predicts whether a track contains no vocals. 'Ooh' and 'aah' sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly 'vocal'. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.")
    print("- Liveness: Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live.")
    print("- Valence: Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).")
    print("- Tempo: the overall estimated tempo of a track in beats per minute (BPM).")
    print("\nInstructions: Adjust the sliders below to match your taste in music:")
    danceability_slider = widgets.FloatSlider(min=0, max=1, step=0.1, description="Danceability:")
    energy_slider = widgets.FloatSlider(min=0, max=1, step=0.1, description="Energy:")
    loudness_slider = widgets.FloatSlider(min=0, max=1, step=0.1, description="Loudness:")
    speechiness_slider = widgets.FloatSlider(min=0, max=1, step=0.1, description="Speechiness:")
    acousticness_slider = widgets.FloatSlider(min=0, max=1, step=0.1, description="Acousticness:")
    instrumentalness_slider = widgets.FloatSlider(min=0, max=1, step=0.1, description="Instrumentalness:")
    liveness_slider = widgets.FloatSlider(min=0, max=1, step=0.1, description="Liveness:")
    valence_slider = widgets.FloatSlider(min=0, max=1, step=0.1, description="Valence:")
    tempo_slider = widgets.FloatSlider(min=0, max=1, step=0.1, description="Tempo:")
    num_songs_input = widgets.IntText(value=10, description="# of songs:")
    button = widgets.Button(description="Create Playlist")
    
    def create_playlist(button):
        num_songs = 10
        danceability = danceability_slider.value
        energy = energy_slider.value
        loudness = loudness_slider.value
        speechiness = speechiness_slider.value
        acousticness = acousticness_slider.value
        instrumentalness = instrumentalness_slider.value
        liveness = liveness_slider.value
        valence = valence_slider.value
        tempo = tempo_slider.value

        song_features = ["Danceability", "Energy", "Loudness_normalized", "Speechiness", "Acousticness",
                     "Instrumentalness", "Liveness", "Valence", "Tempo"]
        weights = [danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo]
        weighted_features = dict(zip(song_features, weights))

        # Selects songs based off user's desired features
        playlist = df[(df[song_features] * weights).sum(axis=1) > 0.5]
        if len(playlist) < num_songs:
            print(f"Not enough songs found that match your desired song features. Your playlist only contains {len(playlist)} unique songs.")
        else:
            num_songs = int(num_songs_input.value)
            playlist = playlist.sample(n=num_songs, replace=True)

        print("\nYour personalized playlist:")
        for i, song in playlist.iterrows():
            print(f"{song['Track']} by {song['Artist']} {song['Url_youtube']}")

    button.on_click(create_playlist)
    display(danceability_slider, energy_slider, loudness_slider, speechiness_slider, acousticness_slider,
            instrumentalness_slider, liveness_slider, valence_slider, tempo_slider, num_songs_input, button)
    
    

Welcome to Smart Jukebox! Answer each question and we'll create a personalized playlist for you.
Would you like to create a playlist that only includes your favorite artist? (yes/no)no

Description of each parameter:
- Danceability: A value of 0.0 is least danceable and 1.0 is most danceable.
- Energy: Death metal has high energy, while a Bach prelude scores low on the scale.
- Loudness: the overall loudness of a track in decibels (dB). 
- Speechiness: The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.
- Acousticness: 1.0 represents high confidence the track is acoustic.
- Instrumentalness: predicts whethe

FloatSlider(value=0.0, description='Danceability:', max=1.0)

FloatSlider(value=0.0, description='Energy:', max=1.0)

FloatSlider(value=0.0, description='Loudness:', max=1.0)

FloatSlider(value=0.0, description='Speechiness:', max=1.0)

...

FloatSlider(value=0.0, description='Valence:', max=1.0)

FloatSlider(value=0.0, description='Tempo:', max=1.0)

IntText(value=10, description='# of songs:')

Button(description='Create Playlist', style=ButtonStyle())


Your personalized playlist:
Consciousness - Eric Prydz Remix by Eric Prydz https://www.youtube.com/watch?v=HwX3iqKUF7A
Jitni Dafa by Jeet Gannguli https://www.youtube.com/watch?v=AvgFmr-ckpk
Home by Toby Fox https://www.youtube.com/watch?v=5_E_y1AWAfc
Lose Yourself to Dance (feat. Pharrell Williams) by Daft Punk https://www.youtube.com/watch?v=NF-kLy44Hls
One More Night (feat. Bryn Christopher) by Benny Benassi https://www.youtube.com/watch?v=eQw_yPYt-Y0
Stand By Me - Remastered 2010 by John Lennon https://www.youtube.com/watch?v=YqB8Dm65X18
Raataan Lambiyan (From "Shershaah") by Tanishk Bagchi https://www.youtube.com/watch?v=gvyUuxdRdR4
All The Pretty Girls by fun. https://www.youtube.com/watch?v=AIoaiTwLk6I
Hoe Cakes by MF DOOM https://www.youtube.com/watch?v=9w1Z3Qq2C8k
Coração Radiante by Grupo Revelação https://www.youtube.com/watch?v=g5Rm1EzQ2gs
